In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [2]:
df=pd.read_csv('Churn_Modelling.csv')

In [3]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df.shape

(10000, 14)

In [5]:
df=df.drop(['RowNumber','CustomerId','Surname'],axis=1)
df.shape

(10000, 11)

In [6]:
X = df.iloc[:,:10]
y = df.iloc[:, 10]

In [7]:
# convert categorical feature into dummy variables

states=pd.get_dummies(X['Geography'],drop_first=True)
gender=pd.get_dummies(X['Gender'],drop_first=True)

In [8]:
#concatenate the remaining dummies columns
X=pd.concat([X,states,gender],axis=1)

#drop the columns as it is no longer required

X=X.drop(['Geography','Gender'],axis=1)

In [9]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [10]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Perform Hyperparameter tuning

In [11]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense,Activation,Embedding,Dropout
from keras.activations import sigmoid,relu

Using TensorFlow backend.


In [36]:
def create_model(layers,activation):
    model=Sequential()
    for i,nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes,input_dim=X_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
            
    model.add(Dense(units=1,init='glorot_uniform',activation='sigmoid'))
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

In [37]:
model=KerasClassifier(build_fn=create_model,verbose=0)

In [40]:
from sklearn.model_selection import RandomizedSearchCV
layers=[[20],[30,15],[45,30,15]]
#layers=[(20,), (30,15), (45,30,15)]
activations=['sigmoid','relu']
p_grid=dict(layers=layers,activation=activations,batch_size=[128,256])
#grid=GridSearchCV(estimator=model,param_grid=p_grid,cv=3)
grid = RandomizedSearchCV(estimator=KerasClassifier(build_fn=create_model,epochs=100,verbose=0), 
                      param_distributions =param_grid)

In [41]:
grid_result= grid.fit(X_train,y_train)

C:\Users\anudeep\anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1, activation="sigmoid", kernel_initializer="glorot_uniform")`
  del sys.path[0]


In [42]:
print(grid_result.best_score_,grid_result.best_params_)

0.8546249985694885 {'layers': (45, 30, 15), 'epochs': 30, 'batch_size': 128, 'activation': 'relu'}


In [43]:
# Predicting the Test set results
y_pred = grid.predict(X_test)
y_pred = (y_pred > 0.5)

In [44]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix,accuracy_score
cm = confusion_matrix(y_test, y_pred)
accuracy=accuracy_score(y_test,y_pred)

In [45]:
print(cm)

[[1540   55]
 [ 217  188]]


In [46]:
print(accuracy)

0.864
